# NBA Standings

In [71]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd 
import numpy as np

years = range(1989,2020,1)
teams = [u'Vancouver Grizzlies','Charlotte Bobcats','Seattle Supersonics','New Orleans Hornets',u'Toronto Raptors', u'Brooklyn Nets', u'Boston Celtics', u'New York Knicks', u'Philadelphia 76ers', u'Chicago Bulls', u'Cleveland Cavaliers', u'Milwaukee Bucks', u'Indiana Pacers', u'Detroit Pistons', u'Atlanta Hawks', u'Washington Wizards', u'Miami Heat', u'Orlando Magic', u'Charlotte Hornets', u'Portland Trail Blazers', u'Oklahoma City Thunder', u'Denver Nuggets', u'Utah Jazz', u'Minnesota Timberwolves', u'Golden State Warriors', u'Los Angeles Clippers', u'Phoenix Suns', u'Sacramento Kings', u'Los Angeles Lakers', u'Memphis Grizzlies', u'Houston Rockets', u'Dallas Mavericks', u'San Antonio Spurs', u'New Orleans Pelicans']
base_url = "https://www.basketball-reference.com/leagues/NBA_{}.html"
nba = pd.DataFrame()

for year in years:
    url = base_url.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    for team in teams:
        try:
            wins = soup.find(text = re.compile(team)).find_next("td").string
            losses = soup.find(text = re.compile(team)).find_next("td").find_next("td").string
            df = pd.DataFrame([team,wins,losses]).T
            df['Year'] = year
            nba = nba.append(df)
        except:
            pass
        
nba.columns = ['Team','Wins','Losses','Year']
nba = nba.reset_index()
nba.drop('index',axis = 1,inplace= True)
none_nba = nba[nba['Wins'].isnull()]
none_nba['Wins'] = none_nba['Losses']
none_nba['Wins'] = none_nba['Wins'].astype('int32')
none_nba['Losses'] = none_nba['Losses'].astype('int32')
none_nba['Losses'] = 82 - none_nba['Wins']


nba.drop(none_nba.index,inplace = True)
nba = nba.append(none_nba)
nba['Wins'] = nba['Wins'].astype('int32')
nba['Losses'] = nba['Losses'].astype('int32')
nba['win%'] = nba['Wins']/(nba['Wins'] + nba['Losses'])

(nba['win%'].std())

# NHL Standings

In [ ]:

years = np.arange(1989,2020,1)
base_url = 'https://www.hockey-reference.com/leagues/NHL_{}_standings.html'

nhl = pd.DataFrame()

for year in years:
    if year == 2005:
        pass
    else:
        req_url = base_url.format(year)
        req = requests.get(req_url)
        soup = BeautifulSoup(req.content, 'html.parser')
        table = soup.find('table', {'id':'standings'})
        df = pd.read_html(str(table))[0]
        df['Year'] = year
        nhl = nhl.append(df[['Unnamed: 1','Overall','Year']])

nhl.columns = ['Team','Overall','Year']
record_nhl = nhl.Overall.str.split("-",expand = True)
record_nhl.columns = ['Wins','RegL','OTL']

record_nhl['RegL'] = record_nhl['RegL'].astype('int32')
record_nhl['OTL'] = record_nhl['OTL'].astype('int32')
record_nhl['Wins'] = record_nhl['Wins'].astype('int32')
record_nhl['Losses'] = record_nhl['RegL'] + record_nhl['OTL']
record_nhl['win%'] = record_nhl['Wins']/(record_nhl['Wins'] + record_nhl['Losses'])

In [96]:
nhlvarob = (record_nhl['win%'].std())**2

0.003048780487804878

In [1]:
nbavarob = nba['win%'].std()**2

NameError: name 'nba' is not defined

In [97]:
nba['win%'].std()**2

0.023426033393787064

In [93]:
.19**2

0.0361

In [109]:
url = 'https://www.basketball-reference.com/playoffs/NBA_2019.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
wins = soup.find_all(id="s874")

In [106]:
url = 'https://www.basketball-reference.com/leagues/NBA_2019.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
wins = soup.find(text = re.compile('Toronto Raptors')).find_next("td").string

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd 
import numpy as np

finals_url = {}
years = range(1989,2020,1)
i = 0


for year in years:
    url = 'https://www.basketball-reference.com/playoffs/NBA_{}.html'
    page = requests.get(url.format(year))
    soup = BeautifulSoup(page.content, 'html.parser')  
    for a in soup.find_all('a', href=True):
        if 'nba-finals-' in a['href']:
            i += 1
            finals_url[i] = a['href']
        else:
            pass

In [2]:
allurls = list(set(finals_url.values()))
allurls.sort()

In [107]:
allurls

['/playoffs/1989-nba-finals-lakers-vs-pistons.html',
 '/playoffs/1990-nba-finals-trail-blazers-vs-pistons.html',
 '/playoffs/1991-nba-finals-lakers-vs-bulls.html',
 '/playoffs/1992-nba-finals-trail-blazers-vs-bulls.html',
 '/playoffs/1993-nba-finals-bulls-vs-suns.html',
 '/playoffs/1994-nba-finals-knicks-vs-rockets.html',
 '/playoffs/1995-nba-finals-rockets-vs-magic.html',
 '/playoffs/1996-nba-finals-supersonics-vs-bulls.html',
 '/playoffs/1997-nba-finals-jazz-vs-bulls.html',
 '/playoffs/1998-nba-finals-bulls-vs-jazz.html',
 '/playoffs/1999-nba-finals-knicks-vs-spurs.html',
 '/playoffs/2000-nba-finals-pacers-vs-lakers.html',
 '/playoffs/2001-nba-finals-76ers-vs-lakers.html',
 '/playoffs/2002-nba-finals-nets-vs-lakers.html',
 '/playoffs/2003-nba-finals-nets-vs-spurs.html',
 '/playoffs/2004-nba-finals-pistons-vs-lakers.html',
 '/playoffs/2005-nba-finals-pistons-vs-spurs.html',
 '/playoffs/2006-nba-finals-heat-vs-mavericks.html',
 '/playoffs/2007-nba-finals-cavaliers-vs-spurs.html',
 '/pl

In [19]:
url = 'https://www.basketball-reference.com/playoffs/1989-nba-finals-lakers-vs-pistons.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')
# wins = soup.find_all(id="four_factors")

In [42]:
url = '/playoffs/1990-nba-finals-trail-blazers-vs-pistons.html'
nba_finals_stats = pd.DataFrame()
bballrefurl = 'https://www.basketball-reference.com{}'
# i = 1
year = 1989

# for url in allurls:

    
def finals_stats(url):
    year = allurls[allurls.index(url)][10:14]
#     i = 1
    print(year)
    finalsurl = bballrefurl.format(url)
    print(finalsurl)
    df = pd.DataFrame()  
    for n in [-1,-2]:
        t1 = pd.read_html(finalsurl)[n]
        t1.columns = t1.columns.get_level_values(1) 
        t1['Team'] = 'Team' + str(n*-1)
        t1['Year'] = year
        df = df.append(t1[['Rk','Player','MP','Team','Year']])
        print(df)
    return df

for url in allurls:
    nba_finals_stats = nba_finals_stats.append(finals_stats(url))
    

1989
https://www.basketball-reference.com/playoffs/1989-nba-finals-lakers-vs-pistons.html
      Rk               Player   MP     MP   Team  Year
0    1.0         James Worthy  170   42.5  Team1  1989
1    2.0  Kareem Abdul-Jabbar  104   26.0  Team1  1989
2    3.0       Michael Cooper  163   40.8  Team1  1989
3    4.0        Tony Campbell   83   20.8  Team1  1989
4    5.0      Mychal Thompson  103   25.8  Team1  1989
5    6.0    Orlando Woolridge   87   21.8  Team1  1989
6    7.0           A.C. Green  134   33.5  Team1  1989
7    8.0        Magic Johnson   75   25.0  Team1  1989
8    9.0         David Rivers   26    8.7  Team1  1989
9   10.0            Jeff Lamp   11    2.8  Team1  1989
10  11.0        Mark McNamara    4    2.0  Team1  1989
11   NaN          Team Totals  960  240.0  Team1  1989
      Rk               Player   MP     MP   Team  Year
0    1.0         James Worthy  170   42.5  Team1  1989
1    2.0  Kareem Abdul-Jabbar  104   26.0  Team1  1989
2    3.0       Michael Cooper 

      Rk           Player    MP     MP   Team  Year
0    1.0  Charles Barkley   277   46.2  Team1  1993
1    2.0    Kevin Johnson   260   43.3  Team1  1993
2    3.0      Dan Majerle   281   46.8  Team1  1993
3    4.0    Richard Dumas   160   26.7  Team1  1993
4    5.0      Danny Ainge   162   27.0  Team1  1993
5    6.0     Tom Chambers    92   15.3  Team1  1993
6    7.0        Mark West   130   21.7  Team1  1993
7    8.0    Oliver Miller   107   17.8  Team1  1993
8    9.0    Frank Johnson    44    7.3  Team1  1993
9   10.0    Jerrod Mustaf     2    1.0  Team1  1993
10   NaN      Team Totals  1515  252.5  Team1  1993
      Rk           Player    MP     MP   Team  Year
0    1.0  Charles Barkley   277   46.2  Team1  1993
1    2.0    Kevin Johnson   260   43.3  Team1  1993
2    3.0      Dan Majerle   281   46.8  Team1  1993
3    4.0    Richard Dumas   160   26.7  Team1  1993
4    5.0      Danny Ainge   162   27.0  Team1  1993
5    6.0     Tom Chambers    92   15.3  Team1  1993
6    7.0    

      Rk            Player    MP     MP   Team  Year
0    1.0       Karl Malone   245   40.8  Team1  1997
1    2.0     John Stockton   226   37.6  Team1  1997
2    3.0     Jeff Hornacek   205   34.2  Team1  1997
3    4.0     Bryon Russell   234   39.0  Team1  1997
4    5.0       Greg Foster    95   15.9  Team1  1997
5    6.0     Howard Eisley    62   10.4  Team1  1997
6    7.0     Greg Ostertag   132   22.0  Team1  1997
7    8.0      Chris Morris    70   11.6  Team1  1997
8    9.0      Antoine Carr    58    9.7  Team1  1997
9   10.0  Shandon Anderson    83   20.8  Team1  1997
10  11.0        Adam Keefe    30    7.4  Team1  1997
11   NaN       Team Totals  1440  240.0  Team1  1997
0    1.0    Michael Jordan   256   42.7  Team2  1997
1    2.0    Scottie Pippen   256   42.7  Team2  1997
2    3.0        Toni Kukoč   139   23.2  Team2  1997
3    4.0        Bison Dele   121   20.1  Team2  1997
4    5.0       Luc Longley   132   22.0  Team2  1997
5    6.0        Ron Harper   162   27.0  Team2

      Rk            Player    MP     MP   Team  Year
0    1.0     Allen Iverson   239   47.8  Team1  2001
1    2.0   Dikembe Mutombo   208   41.6  Team1  2001
2    3.0         Eric Snow   166   33.1  Team1  2001
3    4.0       Aaron McKie   206   41.3  Team1  2001
4    5.0       Tyrone Hill   141   28.2  Team1  2001
5    6.0       Matt Geiger    53   10.7  Team1  2001
6    7.0         Raja Bell    79   15.8  Team1  2001
7    8.0   Todd MacCulloch    31    6.3  Team1  2001
8    9.0     Jumaine Jones    61   12.3  Team1  2001
9   10.0       Kevin Ollie    14    2.9  Team1  2001
10  11.0     Rodney Buford    12    4.1  Team1  2001
11  12.0      George Lynch    13    6.7  Team1  2001
12   NaN       Team Totals  1225  245.0  Team1  2001
0    1.0  Shaquille O'Neal   225   45.0  Team2  2001
1    2.0       Kobe Bryant   234   46.8  Team2  2001
2    3.0      Derek Fisher   156   31.2  Team2  2001
3    4.0          Rick Fox   165   32.9  Team2  2001
4    5.0      Robert Horry   127   25.4  Team2

      Rk            Player    MP     MP   Team  Year
0    1.0  Chauncey Billups   282   40.2  Team1  2005
1    2.0  Richard Hamilton   295   42.2  Team1  2005
2    3.0   Rasheed Wallace   227   32.5  Team1  2005
3    4.0       Ben Wallace   280   40.0  Team1  2005
4    5.0   Antonio McDyess   156   22.3  Team1  2005
5    6.0   Tayshaun Prince   273   39.0  Team1  2005
6    7.0    Lindsey Hunter   137   19.6  Team1  2005
7    8.0     Carlos Arroyo    31    6.3  Team1  2005
8    9.0        Darvin Ham     8    1.5  Team1  2005
9   10.0     Darko Miličić     7    2.3  Team1  2005
10  11.0    Elden Campbell     1    1.1  Team1  2005
11  12.0     Ronald Dupree     9    1.7  Team1  2005
12   NaN       Team Totals  1705  243.6  Team1  2005
      Rk            Player    MP     MP   Team  Year
0    1.0  Chauncey Billups   282   40.2  Team1  2005
1    2.0  Richard Hamilton   295   42.2  Team1  2005
2    3.0   Rasheed Wallace   227   32.5  Team1  2005
3    4.0       Ben Wallace   280   40.0  Team1

      Rk           Player    MP     MP   Team  Year
0    1.0    Hedo Türkoğlu   206   41.1  Team1  2009
1    2.0    Rashard Lewis   212   42.5  Team1  2009
2    3.0    Dwight Howard   213   42.6  Team1  2009
3    4.0     Rafer Alston   147   29.5  Team1  2009
4    5.0  Mickaël Piétrus   140   28.0  Team1  2009
5    6.0     Courtney Lee    88   17.6  Team1  2009
6    7.0      J.J. Redick    65   16.3  Team1  2009
7    8.0    Jameer Nelson    90   18.0  Team1  2009
8    9.0    Marcin Gortat    53   10.7  Team1  2009
9   10.0      Tony Battie    35    7.0  Team1  2009
10   NaN      Team Totals  1250  250.0  Team1  2009
      Rk                Player    MP     MP   Team  Year
0    1.0         Hedo Türkoğlu   206   41.1  Team1  2009
1    2.0         Rashard Lewis   212   42.5  Team1  2009
2    3.0         Dwight Howard   213   42.6  Team1  2009
3    4.0          Rafer Alston   147   29.5  Team1  2009
4    5.0       Mickaël Piétrus   140   28.0  Team1  2009
5    6.0          Courtney Lee    

      Rk          Player    MP     MP   Team  Year
0    1.0      Tim Duncan   254   36.3  Team1  2013
1    2.0     Tony Parker   246   35.2  Team1  2013
2    3.0   Kawhi Leonard   255   36.5  Team1  2013
3    4.0     Danny Green   248   35.4  Team1  2013
4    5.0   Manu Ginóbili   199   28.5  Team1  2013
5    6.0       Gary Neal   171   24.4  Team1  2013
6    7.0  Tiago Splitter   107   15.4  Team1  2013
7    8.0      Boris Diaw    94   15.7  Team1  2013
8    9.0    DeJuan Blair    17    5.8  Team1  2013
9   10.0     Matt Bonner    36    6.1  Team1  2013
10  11.0     Cory Joseph    43    7.1  Team1  2013
11  12.0     Patty Mills    13    6.7  Team1  2013
12  13.0   Nando De Colo     5    2.5  Team1  2013
13  14.0   Tracy McGrady    14    7.2  Team1  2013
14   NaN     Team Totals  1705  243.6  Team1  2013
      Rk          Player    MP     MP   Team  Year
0    1.0      Tim Duncan   254   36.3  Team1  2013
1    2.0     Tony Parker   246   35.2  Team1  2013
2    3.0   Kawhi Leonard   255 

      Rk             Player    MP     MP   Team  Year
0    1.0       LeBron James   212   42.4  Team1  2017
1    2.0       Kyrie Irving   202   40.3  Team1  2017
2    3.0         Kevin Love   161   32.2  Team1  2017
3    4.0         J.R. Smith   146   29.2  Team1  2017
4    5.0  Richard Jefferson    83   16.6  Team1  2017
5    6.0   Tristan Thompson   132   26.4  Team1  2017
6    7.0        Kyle Korver    97   19.3  Team1  2017
7    8.0      Iman Shumpert    66   13.3  Team1  2017
8    9.0      Dahntay Jones    10    3.5  Team1  2017
9   10.0   Derrick Williams    10    3.4  Team1  2017
10  11.0     Deron Williams    61   12.2  Team1  2017
11  12.0      Channing Frye    11   11.2  Team1  2017
12  13.0        James Jones     8    2.8  Team1  2017
13   NaN        Team Totals  1200  240.0  Team1  2017
      Rk                Player    MP     MP   Team  Year
0    1.0          LeBron James   212   42.4  Team1  2017
1    2.0          Kyrie Irving   202   40.3  Team1  2017
2    3.0           

In [44]:
nba_finals_stats.to_csv('nbafinalsstats.csv')


In [17]:
t1 = pd.read_html('https://www.basketball-reference.com/playoffs/1989-nba-finals-lakers-vs-pistons.html')[8]
i += 1
# print(t1)
t1.columns = t1.columns.get_level_values(1) 
# t1['Team'] = 'Team' + str(i)
# t1['Year'] = year
nba_finals_stats = nba_finals_stats.append(t1[['Rk','Player']])
nba_finals_stats

,Rk,Player
0,1.0,Joe Dumars
1,2.0,Isiah Thomas
2,3.0,Vinnie Johnson
3,4.0,James Edwards
4,5.0,Bill Laimbeer
5,6.0,Mark Aguirre
6,7.0,John Salley
7,8.0,Rick Mahorn
8,9.0,Dennis Rodman
9,10.0,John Long


In [52]:
url = 'https://www.hockey-reference.com/playoffs/NHL_2018.html'
page = requests.get(url.format(year))
soup = BeautifulSoup(page.content, 'html.parser')  

for a in soup.find_all('a', href=True):
    if 'stanley' in a['href']:
        print(a['href'])

/playoffs/2018-vegas-golden-knights-vs-washington-capitals-stanley-cup-final.html


In [51]:
pd.read_html('https://www.hockey-reference.com/playoffs/2018-vegas-golden-knights-vs-washington-capitals-stanley-cup-final.html')[-4]

Unnamed: 0_level_0        Unnamed: 1_level_0 Unnamed: 2_level_0  \
                   Rk                    Player                Age   
0                 1.0          William Karlsson               25.0   
1                 2.0     Jonathan Marchessault               27.0   
2                 3.0              David Perron               29.0   
3                 4.0              Reilly Smith               26.0   
4                 5.0                Erik Haula               26.0   
5                 6.0                James Neal               30.0   
6                 7.0              Colin Miller               25.0   
7                 8.0                 Alex Tuch               21.0   
8                 9.0              Nate Schmidt               26.0   
9                10.0             Shea Theodore               22.0   
10               11.0                Cody Eakin               26.0   
11               12.0           Deryk Engelland               35.0   
12               13.0                 Brad Hunt               29.0   
13               14.0  Pierre-Edouard Bellemare               32.0   
14               15.0               Tomas Nosek               25.0   
15               16.0            Brayden McNabb               27.0   
16               17.0            Ryan Carpenter               27.0   
17               18.0                Luca Sbisa               28.0   
18               19.0           Brendan Leipsic               23.0   
19               20.0            Oscar Lindberg               26.0   
20               21.0               Tomas Tatar               27.0   
21               22.0             Brandon Pirri               26.0   
22               23.0           William Carrier               23.0   
23               24.0                Tomas Hyka               24.0   
24               25.0          Jonathon Merrill               25.0   
25               26.0               Ryan Reaves               31.0   
26               27.0          Vadim Shipachyov               30.0   
27               28.0         Marc-Andre Fleury               33.0   
28               29.0            Jason Garrison               33.0   
29               30.0            Stefan Matteau               23.0   
30               31.0               Oscar Dansk               23.0   
31               32.0            Dylan Ferguson               19.0   
32               33.0             Maxime Lagace               25.0   
33               34.0            Malcolm Subban               24.0   
34               35.0           Zach Whitecloud               21.0   
35                NaN                Team Total                NaN   

   Unnamed: 3_level_0 Unnamed: 4_level_0 Scoring           Unnamed: 8_level_0  \
                  Pos                 GP       G    A  PTS                +/-   
0                   C                 82      43   35   78                 49   
1                   C                 77      27   48   75                 36   
2                  LW                 70      16   50   66                  1   
3                  RW                 67      22   38   60                 31   
4                  LW                 76      29   26   55                -16   
5                  LW                 71      25   19   44                -11   
6                   D                 82      10   31   41                 -4   
7                  RW                 78      15   22   37                  3   
8                   D                 76       5   31   36                 19   
9                   D                 61       6   23   29                  5   
10                  C                 80      11   16   27                 -6   
11                  D                 79       5   18   23                  6   
12                  D                 45       3   15   18                 -5   
13                 LW                 72       6   10   16                  5   
14                  C                 67      

In [93]:
nhlfinals_url = 
years = range(1989,2020,1)
i = 0


for year in years:
    url = 'https://www.hockey-reference.com/playoffs/NHL_{}.html'
    page = requests.get(url.format(year))
    soup = BeautifulSoup(page.content, 'html.parser')  
    for a in soup.find_all('a', href=True):
        if 'stanley' in a['href']:
            i += 1
            nhlfinals_url[i] = a['href']
        else:
            pass

In [95]:
allnhlurls = list(set(nhlfinals_url.values()))
allnhlurls.sort()

In [106]:
nhlrefurl = 'https://www.hockey-reference.com{}'
nhl_finals_stats = pd.DataFrame()

def nhlfinals_stats(url):
    print(url)
    year = allnhlurls[allnhlurls.index(url)][10:14]
#     i = 1
    print(year)
    finalsurl = nhlrefurl.format(url)
    print(finalsurl)
    df = pd.DataFrame()  
    for n in [-2,-4]:
        t1 = pd.read_html(finalsurl)[n]
        t1.columns = t1.columns.get_level_values(1) 
        t1['Team'] = 'Team' + str(int(n*-.5))
        t1['Year'] = year
        df = df.append(t1[['Rk','Player','ATOI','Team','Year']])
        print(df)
    return df

for url in allnhlurls:
    nhl_finals_stats = nhl_finals_stats.append(nhlfinals_stats(nhlrefurl))


https://www.hockey-reference.com{}


ValueError: 'https://www.hockey-reference.com{}' is not in list

In [97]:
allnhlurls

['/playoffs/1989-calgary-flames-vs-montreal-canadiens-stanley-cup-final.html',
 '/playoffs/1990-boston-bruins-vs-edmonton-oilers-stanley-cup-final.html',
 '/playoffs/1991-minnesota-north-stars-vs-pittsburgh-penguins-stanley-cup-final.html',
 '/playoffs/1992-chicago-blackhawks-vs-pittsburgh-penguins-stanley-cup-final.html',
 '/playoffs/1993-los-angeles-kings-vs-montreal-canadiens-stanley-cup-final.html',
 '/playoffs/1994-new-york-rangers-vs-vancouver-canucks-stanley-cup-final.html',
 '/playoffs/1995-detroit-red-wings-vs-new-jersey-devils-stanley-cup-final.html',
 '/playoffs/1996-colorado-avalanche-vs-florida-panthers-stanley-cup-final.html',
 '/playoffs/1997-detroit-red-wings-vs-philadelphia-flyers-stanley-cup-final.html',
 '/playoffs/1998-detroit-red-wings-vs-washington-capitals-stanley-cup-final.html',
 '/playoffs/1999-buffalo-sabres-vs-dallas-stars-stanley-cup-final.html',
 '/playoffs/2000-dallas-stars-vs-new-jersey-devils-stanley-cup-final.html',
 '/playoffs/2001-colorado-avalanche-